## 首token时延（TTFT）

在大型语言模型(LLM)的推理过程中，生成第一个token是一个计算密集型任务，这个阶段被称为prefill phase或context phase。这个阶段的计算量较大，也和输入给模型的Prompt的长度呈线性关系。

生成首个token的时间直接影响用户感知的响应速度，因此被称为"首token时延"（Time To First Token, TTFT）。对于服务应用来说，TTFT是衡量用户体验的关键指标，理想情况下应控制在1秒左右。

TTFT受多个因素影响，包括：
1. 模型参数规模
2. 输入Prompt的长度
3. 批处理大小（Batch Size）
4. 可用的GPU资源

为了优化TTFT并提升用户体验，目前大模型厂商开放了两种主要策略：

1. Prompt Cache（提示缓存）：
   通过缓存之前处理过的Prompt结果，我们可以显著减少生成首个token所需的计算时间。这对于处理相似或重复的查询特别有效。

2. Streaming流式输出：
   一旦首个token生成，我们可以立即开始向用户传送结果，而不是等待整个响应完成。这种方法可以大大提升用户感知的响应速度。

这两种技术的结合不仅可以减少用户等待的时间，还能提供更流畅的交互体验。


**OpenAI的Prompt Cache 缓存机制介绍**

1. **基本机制**
- 触发条件：prompt长度超过1024个token时自动启用
- 无需配置：使用completions请求时自动运行
- 最大效果：对10,000+ token的长prompt可降低80%延迟

2. **工作原理**
- 系统会检查prompt的前缀部分是否已缓存
- 如遇缓存命中：直接使用缓存的prompt，减少处理时间和成本
- 如遇缓存未命中：完整处理prompt并缓存前缀供后续使用

3. **重要特性**
- 同一组织共享缓存，同时符合零数据保留要求
- 减少API使用成本（半价）
- 显著降低处理延迟，提升用户体验流畅度

4. **应用场景**
- 代理工具：缓存工具列表和结构化输出模式
- 开发助手：高效处理大量代码库和工作区内容
- 聊天机器人：优化长对话中的静态上下文维护

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241030173235.png)

对于需要频繁与AI系统交互的应用来说，这意味着巨大的经济效益。想象一下，如果一家公司每天需要处理数百万次AI查询，这种成本节约将会产生显著的影响。

**底层原理**

让我用更简单的方式解释 Prompt Cache 的实现原理和过程：

1. 核心思想
想象你在做一份试卷，有些题目你以前做过，已经写好了答案。与其重新思考和计算，直接复制之前的答案更快（注意力状态缓存）。Prompt Cache 就是这个原理：把常用的内容提前"写好答案"存着，需要时直接拿来用。

 2. 主要组成部分

 2.1 模块化系统
- 就像把一本书分成章节，每个章节都有自己的编号（位置ID），可以单独保存和使用
- 即提示标记语言（PML），它允许我们将提示（Prompt）结构化地定义为由多个模块（Module）组成的模式（Schema）。每个模块代表一个可复用的文本片段，例如系统提示、上下文、示例等。
```
<schema name="TaskPrompt">
  <module name="SystemPrompt">...</module>
  <module name="Tools">...</module>
  <module name="Content">...</module>
</schema>
```
- 为每个模块指定一个起始位置 ID。在序列中，各模块的位置 ID 是不连续的，但在模块内部，位置 ID 是连续的。

 2.2 缓存系统
```python
# 简化示意
cache = {
    "常用模块1": "预处理好的结果1",
    "常用模块2": "预处理好的结果2"
}
```

### 3. 工作流程

#### 步骤1：预处理（离线阶段）
- 确定哪些内容经常用到，提前计算好这些内容的结果
- 把结果存在"记忆库"里

#### 步骤2：使用（在线阶段）
1. 查看输入中有哪些是存过的内容
2. 从"记忆库"拿出对应的结果
3. 只需处理新的内容，把所有结果拼在一起

#### 4. 优化效果
- 为什么提升了速度：像是从"抄写"变成了"复制粘贴"
- 为什么价格更低：省资源，不用重复做相同的计算





在这个例子中，我们为客户支持助手定义了工具和交互，能够处理诸如检查交货日期、取消订单和更新付款方式等任务。助手处理两条单独的消息，首先响应初始查询，然后延迟响应后续查询。

缓存工具时，工具定义及其顺序必须保持一致，才能将其包含在 prompt 前缀中。要在多轮对话中缓存消息历史记录，请将新元素附加到 `messages` 数组的末尾。在响应对象和下面的输出中，当看到response的usage的 `cached_tokens` 值大于零，表示缓存成功。我们先看看第一个的案例

In [41]:
import time
import json
from openai import OpenAI

client = OpenAI()  


# 定义工具,包括发货日期查询、订单取消和商品退货的功能模块,
tools = [
   
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date", 
            "description": "获取客户订单的发货日期。当您需要知道发货日期时，请调用此函数，例如当客户询问“我的包裹在哪里”时。",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": { 
                        "type": "string",
                        "description": "客户的订单号。",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_order", 
            "description": "取消尚未发货的订单。当客户请求取消订单时使用此功能。",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "客户的订单号。",
                    },
                    "reason": { 
                        "type": "string",
                        "description": "取消订单的原因。",
                    }
                },
                "required": ["order_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "return_item", 
            "description": "处理订单退货。当客户想要退货且订单已交付时，应调用此函数。",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": { 
                        "type": "string",
                        "description": "客户的订单号。",
                    },
                    "item_id": { 
                        "type": "string",
                        "description": "客户要退回的特定商品号。",
                    },
                    "reason": { 
                        "type": "string",
                        "description": "退货的原因。",
                    }
                },
                "required": ["order_id", "item_id", "reason"],
                "additionalProperties": False
            }
        }
    },

]
# 增强版系统消息，带有保护措施
messages = [
    {
        "role": "system",
        "content": (
            "你是一位专业、热情且高效的客服助理。你的任务是以热情友好的态度为客户提供快速、清晰、全面的帮助。"
            "时刻表达同理心，尤其是在用户感到沮丧或担忧时，并确保你的语言礼貌且专业。"
            "使用简单明了的沟通方式避免任何误解，并在采取行动前与用户确认。"
            "在更复杂或时间紧迫的情况下，向用户保证你正在迅速采取行动，并定期提供更新。"
            "适应用户的语气：即使在压力大或困难的情况下，也要保持冷静、友好和理解。"
            "\n\n"
            "此外，在帮助用户时，你必须遵守以下几个重要的保护措施："
            "\n\n"
            "1. **保密性和数据隐私**：不要泄露任何关于公司或其他用户的敏感信息。处理个人信息（例如订单号、地址或付款方式）时，"
            "请确保以最高级别的保密性对待这些信息。如果用户请求访问其数据，请仅提供与其请求相关的必要信息，确保不会意外泄露其他用户信息。"
            "\n\n"
            "2. **安全支付处理**：更新付款详细信息或处理退款时，始终确保支付数据（例如信用卡号、CVV 码和到期日期）的安全传输和存储。"
            "切勿显示或记录完整的信用卡号。在处理任何付款更改或退款之前，请与用户确认。"
            "\n\n"
            "3. **尊重界限**：如果用户表达了沮丧或不满，请保持冷静和同理心，但避免越过专业的界限。不要做出个人判断，"
            "避免使用可能加剧局势的语言。坚持事实信息和明确的解决方案来解决用户的问题。"
            "\n\n"
            "4. **遵守法律**：确保你采取的所有行动都符合法律法规标准。例如，如果用户要求退款、取消或退货，请严格遵守公司的退款政策。"
            "如果由于已发货或其他限制而无法取消订单，请清晰而同情地解释政策。"
            "\n\n"
            "5. **一致性**：始终提供与公司政策一致的信息。如果不确定公司政策，请与用户沟通清楚，告知他们你正在核实信息，避免做出虚假承诺。"
            "如果将问题升级到其他团队，请告知用户，并提供他们可以预期得到解决方案的实际时间表。"
            "\n\n"
            "6. **用户赋能**：尽可能让用户做出明智的决定。向他们提供相关选项，并清楚地解释每个选项，确保他们理解每个选择的后果"
            "（例如，取消订单可能会导致失去积分等）。确保你的帮助支持他们的自主性。"
            "\n\n"
            "7. **不作推测**：不要对结果进行推测或提供你不确定的信息。在讨论订单状态、政策或潜在解决方案时，始终坚持已核实的事实。"
            "如果不清楚，请告诉用户你将在做出任何承诺之前进行进一步调查。"
            "\n\n"
            "8. **尊重和包容的语言**：无论用户的语气如何，请确保你的语言保持包容性和尊重性。避免基于有限的信息做出假设，并注意用户的不同需求和背景。"
        )
    },
    {
        "role": "user",
        "content": (
            "你好，我三天前下了订单，但还没有收到任何关于何时发货的更新。"
            "你能帮我查一下发货日期吗？我的订单号是 #9876543210。我有点担心，因为我急需这个商品。"  
        )
    }
]

# 增强版 user_query2
user_query2 = {
    "role": "user",
    "content": (
        "由于我的订单实际上还没有发货，我想取消它。"
        "订单号是 #9876543210，我需要取消是因为我决定在本地购买，以便更快地拿到它。"  
        "你能帮我取消吗？谢谢！"
    )
}
# 使用提供的消息历史记录和工具运行
def completion_run(messages, tools): 
    completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        tools=tools,
        messages=messages,
    )
    usage_data = json.dumps(completion.to_dict(), indent=4, ensure_ascii=False)
    return usage_data

# 主函数，用于处理两次运行
def main(messages, tools, user_query2): 
    # 运行 1：初始查询
    print("运行 1:") 
    run1 = completion_run(messages, tools)
    print(run1)

    # 延迟 7 秒
    time.sleep(7)

    # 将 user_query2 追加到消息历史记录
    messages.append(user_query2)

    # 运行 2：附加查询
    print("\n运行 2:") 
    run2 = completion_run(messages, tools)
    print(run2)


# 运行主函数
main(messages, tools, user_query2)

运行 1:
{
    "id": "chatcmpl-ANdmOdsgOq3RocMLS10MW3Vkohova",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_ohL52VAJginfM7tf9VYxAxVG",
                        "function": {
                            "arguments": "{\"order_id\":\"9876543210\"}",
                            "name": "get_delivery_date"
                        },
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1730197820,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "system_fingerprint": "fp_8bfc6a7dc2",
    "usage": {
        "completion_tokens": 19,
        "prompt_tokens": 926,
        "total_tokens": 945,
        "comp

可以看到缓存token为0 不是说自动启用缓存吗 这是为什么没有缓存上呢？

噢 原来是prompt token数没有到1024个呀，我们再加几个工具描述看看缓存情况

In [44]:
import time
import json
from openai import OpenAI

client = OpenAI()  


# 定义工具,包括发货日期查询、订单取消和商品退货的功能模块。加入订单状态查询，生成订单发票，更新配送地址功能
tools = [
     {
    "type": "function",
    "function": {
        "name": "get_order_status", 
        "description": "查询客户订单的当前状态。",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": { 
                    "type": "string",
                    "description": "客户的订单号。",
                },
            },
            "required": ["order_id"],
            "additionalProperties": False,
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "update_delivery_address", 
        "description": "在订单未发货时更新客户的配送地址。",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "客户的订单号。",
                },
                "new_address": { 
                    "type": "string",
                    "description": "新的配送地址。",
                }
            },
            "required": ["order_id", "new_address"],
            "additionalProperties": False
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "generate_invoice", 
        "description": "生成订单的详细发票。",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": { 
                    "type": "string",
                    "description": "客户的订单号。",
                }
            },
            "required": ["order_id"],
            "additionalProperties": False
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "provide_discount_code", 
        "description": "根据客户要求或活动提供优惠码。",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "客户的ID号。",
                },
                "discount_type": { 
                    "type": "string",
                    "description": "优惠类型，例如“新客户”、“节日活动”等。",
                }
            },
            "required": ["customer_id", "discount_type"],
            "additionalProperties": False
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "update_item_quantity", 
        "description": "在订单处理中更新物品数量。",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": { 
                    "type": "string",
                    "description": "客户的订单号。",
                },
                "item_id": { 
                    "type": "string",
                    "description": "需要更改数量的商品号。",
                },
                "quantity": {
                    "type": "integer",
                    "description": "新的物品数量。",
                }
            },
            "required": ["order_id", "item_id", "quantity"],
            "additionalProperties": False
        }
    }
},
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date", 
            "description": "获取客户订单的发货日期。当您需要知道发货日期时，请调用此函数，例如当客户询问“我的包裹在哪里”时。",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": { 
                        "type": "string",
                        "description": "客户的订单号。",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_order", 
            "description": "取消尚未发货的订单。当客户请求取消订单时使用此功能。",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "客户的订单号。",
                    },
                    "reason": { 
                        "type": "string",
                        "description": "取消订单的原因。",
                    }
                },
                "required": ["order_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "return_item", 
            "description": "处理订单退货。当客户想要退货且订单已交付时，应调用此函数。",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": { 
                        "type": "string",
                        "description": "客户的订单号。",
                    },
                    "item_id": { 
                        "type": "string",
                        "description": "客户要退回的特定商品号。",
                    },
                    "reason": { 
                        "type": "string",
                        "description": "退货的原因。",
                    }
                },
                "required": ["order_id", "item_id", "reason"],
                "additionalProperties": False
            }
        }
    },

]
# 增强版系统消息，带有保护措施
messages = [
    {
        "role": "system",
        "content": (
            "你是一位专业、热情且高效的客服助理。你的任务是以热情友好的态度为客户提供快速、清晰、全面的帮助。"
            "时刻表达同理心，尤其是在用户感到沮丧或担忧时，并确保你的语言礼貌且专业。"
            "使用简单明了的沟通方式避免任何误解，并在采取行动前与用户确认。"
            "在更复杂或时间紧迫的情况下，向用户保证你正在迅速采取行动，并定期提供更新。"
            "适应用户的语气：即使在压力大或困难的情况下，也要保持冷静、友好和理解。"
            "\n\n"
            "此外，在帮助用户时，你必须遵守以下几个重要的保护措施："
            "\n\n"
            "1. **保密性和数据隐私**：不要泄露任何关于公司或其他用户的敏感信息。处理个人信息（例如订单号、地址或付款方式）时，"
            "请确保以最高级别的保密性对待这些信息。如果用户请求访问其数据，请仅提供与其请求相关的必要信息，确保不会意外泄露其他用户信息。"
            "\n\n"
            "2. **安全支付处理**：更新付款详细信息或处理退款时，始终确保支付数据（例如信用卡号、CVV 码和到期日期）的安全传输和存储。"
            "切勿显示或记录完整的信用卡号。在处理任何付款更改或退款之前，请与用户确认。"
            "\n\n"
            "3. **尊重界限**：如果用户表达了沮丧或不满，请保持冷静和同理心，但避免越过专业的界限。不要做出个人判断，"
            "避免使用可能加剧局势的语言。坚持事实信息和明确的解决方案来解决用户的问题。"
            "\n\n"
            "4. **遵守法律**：确保你采取的所有行动都符合法律法规标准。例如，如果用户要求退款、取消或退货，请严格遵守公司的退款政策。"
            "如果由于已发货或其他限制而无法取消订单，请清晰而同情地解释政策。"
            "\n\n"
            "5. **一致性**：始终提供与公司政策一致的信息。如果不确定公司政策，请与用户沟通清楚，告知他们你正在核实信息，避免做出虚假承诺。"
            "如果将问题升级到其他团队，请告知用户，并提供他们可以预期得到解决方案的实际时间表。"
            "\n\n"
            "6. **用户赋能**：尽可能让用户做出明智的决定。向他们提供相关选项，并清楚地解释每个选项，确保他们理解每个选择的后果"
            "（例如，取消订单可能会导致失去积分等）。确保你的帮助支持他们的自主性。"
            "\n\n"
            "7. **不作推测**：不要对结果进行推测或提供你不确定的信息。在讨论订单状态、政策或潜在解决方案时，始终坚持已核实的事实。"
            "如果不清楚，请告诉用户你将在做出任何承诺之前进行进一步调查。"
            "\n\n"
            "8. **尊重和包容的语言**：无论用户的语气如何，请确保你的语言保持包容性和尊重性。避免基于有限的信息做出假设，并注意用户的不同需求和背景。"
        )
    },
    {
        "role": "user",
        "content": (
            "你好，我三天前下了订单，但还没有收到任何关于何时发货的更新。"
            "你能帮我查一下发货日期吗？我的订单号是 #9876543210。我有点担心，因为我急需这个商品。"  
        )
    }
]

# 增强版 user_query2
user_query2 = {
    "role": "user",
    "content": (
        "由于我的订单实际上还没有发货，我想取消它。"
        "订单号是 #9876543210，我需要取消是因为我决定在本地购买，以便更快地拿到它。"  
        "你能帮我取消吗？谢谢！"
    )
}
# 使用提供的消息历史记录和工具运行
def completion_run(messages, tools): 
    completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        tools=tools,
        messages=messages,
    )
    usage_data = json.dumps(completion.to_dict(), indent=4, ensure_ascii=False)
    return usage_data

# 主函数，用于处理两次运行
def main(messages, tools, user_query2): 
    # 运行 1：初始查询
    print("运行 1:") 
    run1 = completion_run(messages, tools)
    print(run1)

    # 延迟 7 秒
    time.sleep(7)

    # 将 user_query2 追加到消息历史记录
    messages.append(user_query2)

    # 运行 2：附加查询
    print("\n运行 2:") 
    run2 = completion_run(messages, tools)
    print(run2)


# 运行主函数8
main(messages, tools, user_query2)

运行 1:
{
    "id": "chatcmpl-ANe3v11KDQKeKbBHDhgSO7sBAZnVf",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_vIsktwqe536T0sOufJO7n4ld",
                        "function": {
                            "arguments": "{\"order_id\":\"9876543210\"}",
                            "name": "get_delivery_date"
                        },
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1730198907,
    "model": "gpt-4o-mini-2024-07-18",
    "object": "chat.completion",
    "system_fingerprint": "fp_f59a81427f",
    "usage": {
        "completion_tokens": 19,
        "prompt_tokens": 1136,
        "total_tokens": 1155,
        "co

自动缓存成功了！这1024个token是半价噢！



为了充分利用 prompt 缓存，请考虑遵循以下最佳实践：

- **将静态或经常重复使用的内容放在 prompt 的开头：** 这有助于通过将动态数据保留在 prompt 的末尾来确保更好的缓存效率。
- **保持一致的使用模式：** 不定期使用的 prompt 会自动从缓存中移除。为了防止缓存逐出，请保持一致地使用 prompt。
* **监控关键指标：** 定期跟踪缓存命中率、延迟和缓存 token 的比例。使用这些洞察力来微调您的缓存策略并最大限度地提高性能。


上面这种业务场景机器人常常会产生幻觉，即它觉得它可以实际上不能完成客户要求。

有一些解决方法如微调，加入一些问题和正确答案应该是拒绝的数据语料进行微调，或这种prompt，但机器幻觉仍是个难以杜绝的场景。

下面prompt是改善类似业务场景中机器幻觉的prompt

In [ ]:
PROMPT = """\
你正在将提交的答案与给定问题的专家答案进行比较。以下是数据：
[开始数据]
************
[问题]: {{input}}
************
[专家答案]: {{expected}}
************
[提交答案]: {{output}}
************
[结束数据]

比较提交答案与专家答案的事实内容。忽略任何风格、语法或标点符号的差异。
提交的答案可能是专家答案的子集或超集，或者可能与其存在冲突。确定适用哪种情况。请从以下选项中选择一个来回答问题：
(A) 提交的答案是专家答案的子集，并且与其完全一致。
(B) 提交的答案是专家答案的超集，并且与其完全一致。
(C) 提交的答案包含与专家答案相同的所有细节。
(D) 提交的答案与专家答案之间存在分歧。
(E) 答案有所不同，但从事实性的角度来看这些差异并不重要。

请通过调用`select_choice`函数来回答问题，以步骤化的方式说明你的推理过程，以确保你的结论是正确的。避免在一开始就直接陈述正确答案。通过将`choice`参数设置为A、B、C、D或E中的一个选项来选择单一答案。
"""


choice_scores={"A": 0.5, "B": 0, "C": 1, "D": 0, "E": 1},


1. 对话型助手:

通过缓存长指令或上传的文档,可以降低成本并加快长对话的响应速度。
实际应用:客服聊天机器人可以预先加载产品手册、常见问题等信息,从而更快速准确地回答客户问题。

2. 编程助手:

通过在提示中保留代码库的摘要版本,改善代码自动补全和问答功能。
实际应用:IDE插件可以分析整个项目代码,为开发者提供更智能的代码建议和上下文相关的帮助。

3. 大文档处理:

在提示中包含完整的长篇材料(包括图像),而不增加响应延迟。
实际应用:法律文件审核系统可以一次性处理整份合同,快速找出关键条款或潜在风险。

4. 详细指令集:

通过共享大量指令、流程和示例来微调AI的响应。
实际应用:在教育平台上,可以为AI助教提供大量优秀作文范例,使其能更准确地评分和提供反馈。

5. 主动搜索和工具使用:

提高需要多轮工具调用和迭代修改场景的性能。
实际应用:智能家居系统可以根据用户习惯、天气预报等信息,主动调整家电设置,无需频繁请求服务器。

6. 与长篇内容对话:

将整个文档嵌入提示中,让用户能与任何知识库进行问答交互。
实际应用:企业可以将所有内部文档、培训材料输入AI系统,员工可以随时查询,获得精准答案。